Dr Oliviero Andreussi, olivieroandreuss@boisestate.edu

Boise State University, Department of Chemistry and Biochemistry

# Fitting and Data Analysis for the Differential Scanning Calorimetry Experiment {-}

## Preliminary Setup {-}

Before we start, let us import the main modules that we will need for this lecture. You may see some new modules in the list below, we will add more details in the right sections.

In [ ]:
# @title Notebook Setup { display-mode: "form" }
# Import the main modules used in this worksheet
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from scipy.signal import medfilt
from scipy.integrate import simps
# Load the google drive with your files 
#from google.colab import drive
#drive.mount('/content/drive')
# The following needs to be the path of the folder with all your datafile in .csv format
base_path = '/content/drive/MyDrive/'

In [ ]:
# @title Functions to load the data { display-mode: "form" }
def load_data_to_file_dict(file_dict):
    """
    Load a DSC .txt file. 
    The format of the file should have two rows of headers, one extra line of text at the end, and five columns of data
    The columns are: index, time, heat flow, temperature of sample, temperature of reference

    Input variables:
        file_dict : a dictionary with 'path' and 'name' keys corresponding to the file to be loaded
    
    Action: 
        Add to file_dict a Pandas DataFrame with four columns: time (in seconds), heat flow (in W/g), 
        temperature of sample (in C), and temperature of reference (in C)
    """
    data = pd.read_csv(file_dict['path']+file_dict['name'],skiprows=2,skipfooter=1,names=['Time','Heat-Flow','Ts','Tr'],sep=' +',index_col=0,engine='python') 
    file_dict['data'] = data
    return

def plot_peak(peak_dict,xaxis='Time'):
    """ 
    """
    time_start = peak_dict['time_start']
    time_end = peak_dict['time_end']
    if time_end == 0 : time_end = peak_dict['data'].Time.iat[-1]
    # 
    filtered_data = peak_dict['data'].query(f'Time > {time_start} and Time < {time_end}')
    #
    baseline=medfilt(filtered_data['Heat-Flow'],peak_dict['baseline_width'])
    #
    plt.plot(filtered_data[xaxis],filtered_data['Heat-Flow'])
    plt.plot(filtered_data[xaxis],baseline)
    plt.ylabel('Heat Flow [W/g]')
    if xaxis not in peak_dict['data']:
        print("ERROR: Invalid value for xaxis argument")
        return
    elif xaxis == 'Time' :
        plt.xlabel('Time [s]')
    elif xaxis == 'Ts' :
        plt.xlabel('Ts [C]')
    elif xaxis == 'Tr' :
        plt.xlabel('Tr [C]')
    plt.show()


In [ ]:
# @title Functions to fit the data { display-mode: "form" }
def analyze_peak(peak_dict,verbose=False):
    """ 
    """
    if not ('data' in peak_dict): 
        load_data_to_file_dict(peak_dict)
    # Clean signal (filter and baseline)
    time_start = peak_dict['time_start']
    time_end = peak_dict['time_end']
    if time_end == 0 : time_end = peak_dict['data'].Time.iat[-1]
    # 
    filtered_data = peak_dict['data'].query(f'Time > {time_start} and Time < {time_end}').copy()
    #
    baseline=medfilt(filtered_data['Heat-Flow'],peak_dict['baseline_width'])
    filtered_data['Heat-Flow-Clean'] = filtered_data['Heat-Flow'] - baseline
    # Compute the integral
    peak_integral = simps(filtered_data['Heat-Flow-Clean'],filtered_data['Ts'])
    peak_dict['integral'] = peak_integral
    if verbose : print(f'The peak integral is {peak_integral:8.6f}')
    # Compute the second derivative of the Heat-Flow wrt Time
    first_derivative = np.gradient(filtered_data['Heat-Flow'], filtered_data['Time'])
    second_derivative = np.gradient(first_derivative, filtered_data['Time'])
    if peak_integral < 0 : 
        # use the minimum value
        transition_index = np.argmax(second_derivative)
    else:
        # use the maximum value
        transition_index = np.argmin(second_derivative)
    transition_temperature = filtered_data.Ts.iat[transition_index]
    peak_dict['T_transition'] = transition_temperature
    if verbose : 
        print(f'The transition temperature is {transition_temperature:6.4f}')
        plt.plot(filtered_data['Ts'],second_derivative)
        plt.show()



## Visualize the Systems {-}

The following module needs to be installed on Colab. We won't need it too much for this analysis, but they offer a lot of nice features for chemistry programming. 

In [ ]:
# @title Install and load RDKit { display-mode: "form" }
!pip install rdkit
from rdkit import Chem
from rdkit.Chem import Draw
!pip install cirpy
import cirpy

In particular we can use them to draw the molecules in our experiments. While for some molecules you can just write their names and RDKit will plot them, for most molecules you will need to provide their SMILES or their CAS numbers. Luckily, CIRpy can usually find SMILES for you, if you type the common name correctly or if you know the CAS number. 

These are the SMILES for the molecules in your DSC experiments:
* paracetamol: 'CC(=O)Nc1ccc(O)cc1'

In [ ]:
# @title Choose the molecule to draw { display-mode: "form" }
input = 'paracetamol' # @param {type:"string"}
input_type = 'name' # @param ["smiles", "name", "cas"] {allow-input: true}
if input_type != 'smiles' :
    smiles=cirpy.resolve( input, 'smiles')
else:
    smiles=input
img = Draw.MolToImage( Chem.MolFromSmiles(smiles), size=(300, 300) )
display(img)

## Loading the Data {-}

We need to load the Google Drive and access an example of a dataset from a kinetics experiment. You can use the same set that I am using by downloading it from Canvas, [here](). Or you can use your own files. I am assuming the file in question will be located in a `Kinetics_Data/` subfolder in your `Colab Notebook/` folder. 

In [ ]:
# @title Set Local Path { display-mode: "form" }
# The following needs to be the path of the folder with all your collected data in .csv format
local_path="Colab Notebooks/DSC_Data/" # @param {type:"string"}
path = base_path+local_path

In order to streamline the fitting of the data, we will be storing the file that corresponds to each experiment into a Python dictionary (`dict`), together with all the relevant information of that experiment and the parameters that we need for the fit. You can use the same statement in the following, but make sure to change the file name from 'indium.txt' (the one that I am using) to the one you want to use.

In [ ]:
indium_melting = {'path':path, 'name':'indium.txt', 'label':'Indium', 'time_start': 0, 'time_end': 0, 'baseline_width': 61}

In [ ]:
load_data_to_file_dict(indium_melting)

We should always check that the loaded data looks ok, say by checking the number of columns and rows and, maybe, plotting the two columns

In [ ]:
indium_melting['data'].info()

In [ ]:
# @title Find a reasonable time interval around the melting peak { display-mode: "form" }
time_start = 250 # @param {type:"number"}
time_end = 350 # @param {type:"number"}
baseline_width = 61 # @param {type:"integer"}
indium_melting['time_start'] = time_start
indium_melting['time_end'] = time_end
indium_melting['baseline_width'] = baseline_width
plot_peak(indium_melting,xaxis='Time')

In [ ]:
# @title Find the transition temperature and the peak integral { display-mode: "form" }
analyze_peak(indium_melting,verbose=True)